<a href="https://colab.research.google.com/github/aniass/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [12]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'), 
  ])


In [26]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 2.6864 - accuracy: 0.2635 - val_loss: 1.3188 - val_accuracy: 0.5882
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.9950 - accuracy: 0.6754 - val_loss: 0.2853 - val_accuracy: 0.9208
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4332 - accuracy: 0.8645 - val_loss: 0.1149 - val_accuracy: 0.9658
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2659 - accuracy: 0.9178 - val_loss: 0.0597 - val_accuracy: 0.9816
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1883 - accuracy: 0.9435 - val_loss: 0.0467 - val_accuracy: 0.9863


0.963718820861678

In [27]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1299 - accuracy: 0.9637


0.963718831539154

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose=0,
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'acuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}

In [17]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20)
}

best = fmin(
     func_obj,
     space,
     tpe.suggest,
     1,
     Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.4976 - accuracy: 0.8750

 15/138 [==>...........................]
 - ETA: 0s - loss: 0.1882 - accuracy: 0.9396

 28/138 [=====>........................]
 - ETA: 0s - loss: 0.3339 - accuracy: 0.9230

 41/138 [=======>......................]
 - ETA: 0s - loss: 0.2378 - accuracy: 0.9459

 56/138 [===========>..................]
 - ETA: 0s - loss: 0.2280 - accuracy: 0.9425

 68/138 [=============>................]
 - ETA: 0s - loss: 0.2032 - accuracy: 0.9490


In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'), 
  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'acuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}

In [37]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
     func_obj,
     space,
     tpe.suggest,
     30,
     Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.44913139891056, 'dropout_cnn_block_three': 0.4351934998426168, 'dropout_cnn_block_two': 0.30000203887292465, 'dropout_dense_block_one': 0.5165197652735445, 'dropout_dense_block_two': 0.36351640778484406}
acuracy=0.9585034251213074
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.162111598164966, 'dropout_cnn_block_three': 0.5665457577929074, 'dropout_cnn_block_two': 0.5473844840042497, 'dropout_dense_block_one': 0.12982698468728693, 'dropout_dense_block_two': 0.693406778555555}
acuracy=0.9158729910850525
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.2874100569097271, 'dropout_cnn_block_three': 0.6144583173155236, 'dropout_cnn_block_two': 0.8711989184849889, 'dropout_dense_block_one': 0.6308289965029704, 'dropout_dense_block_two': 0.7415386127914458}
acuracy=0.2351473867893219
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.9422073015614268, 'dropout_cnn_block_three': 0.593600479977167, 'dropout_cnn_block_two': 0.4198098224063992, 'dr

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'acuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}

In [34]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
     func_obj,
     space,
     tpe.suggest,
     30,
     Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.44216643521107346, 'dropout_cnn_block_three': 0.3199064559841065, 'dropout_cnn_block_two': 0.4521104933344844, 'dropout_dense_block_one': 0.6095002044406883, 'dropout_dense_block_two': 0.6843485808376654}
acuracy=0.8281179070472717
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.42454109647939764, 'dropout_cnn_block_three': 0.39858940734635756, 'dropout_cnn_block_two': 0.4556919279827045, 'dropout_dense_block_one': 0.5595311568794712, 'dropout_dense_block_two': 0.5856764979669327}
acuracy=0.922448992729187
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.39529883386448084, 'dropout_cnn_block_three': 0.3955911751416832, 'dropout_cnn_block_two': 0.38438703519386297, 'dropout_dense_block_one': 0.5455977583219191, 'dropout_dense_block_two': 0.39158395293066195}
acuracy=0.9376417398452759
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.32926246209710636, 'dropout_cnn_block_three': 0.3476005647746331, 'dropout_cnn_block_two': 0.4464228335498